In [9]:
#Calculate how different two sequences are from each other. The bigger the number, the more different
#p and q are both sequence strings that should be the same length
#Return an integer
def HammingDistance(p, q):
    dist = 0
    if len(p)==len(q): #check to make sure sequences are the same length
        for i in range(len(p)):
            if p[i]==q[i]: #if the sequence has the same letter at the same position, don't add to score
                dist+=0
            else:
                dist+=1
    return dist

In [10]:
def Neighbors(pattern, d):
    if d == 0: #if you're allowed no mismatches, just return the pattern
        return pattern
    elif len(pattern) == 1: #if the pattern is only 1nt long, return all the possible nucleotides
        return {'A', 'C', 'G', 'T'}
    neighborhood = []
    suffix_neighbors = Neighbors(pattern[1:], d) #recursion. Logically goes off the rails for me here.
    for text in suffix_neighbors:
        nucleotides = {'A', 'C', 'G', 'T'}
        if HammingDistance(pattern[1:], text) < d:
            for x in nucleotides:
                neighborhood.append(x + text)
        else:
            neighborhood.append(pattern[0] + text)
    return neighborhood

In [14]:
def MotifEnumeration(dna, k, d):
    motifs = []
    for i in range(len(dna)): #cycle through the dna list
        if i == 0: #for the first string
            for j in range(len(dna[i])-k+1): #cycle through the first string
                pattern = (dna[i][j:j+k]) #identify a sequence of k-length in the first string
                pattern_p=Neighbors(pattern, d) #use that pattern to identify all possibilities with d mismatches
                if isinstance(pattern_p, str)==True:  #If there's only one value, it's a string not a list.
                    motifs.append(pattern_p) #add to potential motif list
                else:
                    motifs.extend(pattern_p) #add to potential motif list
    motifs = list(set(motifs)) #get rid of duplicates
    temp = {}
    removemotif = []
    for j in range(1, len(dna)): #now look at the other dna strings
        for motif in motifs: #cycle through the motifs and see if they're in the other dna strings
            #print(motif) #checking that things are working
            for i in range(len(dna[j])-k+1):
                if HammingDistance(motif, dna[j][i:i+k]) <= d: #identify if motif occurs in the string
                    if motif in list(temp.keys()):
                        temp[motif]+=1
                    else:
                        temp[motif]=1
                else:
                    if motif not in list(temp.keys()):
                        temp[motif]=0
            if temp[motif] == 0: #identify motifs that should be removed because they don't occur in the string
                removemotif.append(motif)
        #print(motifs) #checking that things are working
        temp = {} #reset temp before cycling to the next dna string
    for motif in list(set(removemotif)): #remove all the motifs not in every string
        motifs.remove(motif)
    return motifs

text = ["CTTTGCTCGAAACCCTGTGGACTTC", "CATGGTTAATTACCTAAACCCGTTC", "CTATTGTGTCAAACCCCCGGGTGCG", "ACGGGTGTGCACCGCCAACTCATCC", 
        "AATGATCGTAGATCCGAGCCGCATT", "TGCTACACCCGGATTCGTGAAGGGA"]
a = MotifEnumeration(text, 5, 2)
print(*a)

TAACC TAGGT AGGAT TTTTC TGGCT AGCCA TGAGC CGGCC ATACC AGTAC CTATT GTAAC TGCGT ACGGC GTTCT GGTGG ATCGG CGATG TAGCC TTGAC TCGCC TGTAA AAAGT TTGTT CGAAC ATCGT TGTGC GAACA GTGGC AGTGA AATTT CCCGT CGGAT TGGAA TACGG ATTCC GTTGA ATGCA TATAC CGCTC CCATC GGTCC GAGAA CGGTT GGAAC CGGTG CCTAA CATAT ACGGT GGTTG ACCGT TTATG GGATT TGTAC GGATA CCAGT GATCA TGCCT GAATG TTCAT CCAAC CCGAT CTGAT CCTCC CGATA GCTCG GATGG GTCGT ACACC CCCAG TCCGT TAAAC ATGAG AGGCC AGCGC CCCAA GAAAA AGTTG ACCTT GCGTT CATTC CTGGC GGAAA AGTTC ATCAT GGGTT GATAA GGGTC GTGCA GCCTA AAGCA CTACA CAGTG CCTCG TGGGC TGTGA CCTGA GCCAA CTCCG GGAGA CCAAG GGACC GCCGG GCCGT TGAGT TCCGG AATCA ACCAG AAGGC CCACG TCAGC CACAC TACTA TCCTC CGCTG ACCCA TTTGT TCTTA CCACT AAAGG CTGCA TAACG ATACT ACGTG GATAC AAGTC ATCAA GGGAT CTTCG TATCC TTGCA GGTAA TCATG TTCCC GTAAG TACTC GGTCA GAACG CTTCC CCATA CGGGC ACATA GACCA GTCAA TTCTC CGTCT ATTTC CATAC CTCCC CTGCC AAATC ACTCG ACAAC GACGC TTCTT GTTTT CCCCT AGGGA CTTAA CGTAT AACTA AGTTA TATGT TCCAC CTTAC AGACC GACA

In [15]:
#Count how frequently a symbol occcurs at a certain position when given a list of motifs of the same length
#motifs is a list of strings of the same length
#count output is a dictionary with A C G T as the keys
def Count(motifs):
    count = {} # initializing the count dictionary
    k = len(motifs[0]) #get the length of each motif row (x axis)
    for symbol in "ACGT": #Make A C G T a key
        count[symbol] = [] #make an empty list for each key
        for j in range(k): #for the length of the motif...
            count[symbol].append(0) #...for the key, put a zero for each position in the list for now
    t = len(motifs) #get the number of keys in the dictionary (y axis)
    for i in range(t): #for every key...
        for j in range(k): #for list position in each key...
            symbol = motifs[i][j] #for this specific position for a key...
            count[symbol][j] += 1 #Add one every time this position occurs in the matrix at this position
    return count

In [18]:
#similar to Count, but get a ratio instead. All columns should add to 1.
#motifs input is a list of strings of the same length
#profile output is a dictionary with A C G T as keys
def Profile(motifs):
    t = len(motifs)
    k = len(motifs[0])
    profile = Count(motifs)
    sub= Count(motifs)
    for x in range(t): #for key in position x...
        for y in range(k): #for list position in that key...
            symbol=motifs[x][y]
            profile[symbol][y]=(sub[symbol][y])/t
    return profile

motifs = [
"TCGGGGGTTTTT",
"CCGGTGACTTAC",
"ACGGGGATTTTC",
"TTGGGGACTTTT",
"AAGGGGACTTCC",
"TTGGGGACTTCC",
"TCGGGGATTCAT",
"TCGGGGATTCCT",
"TAGGGGAACTAC",
"TCGGGTATAACC"
]

Profile(motifs)

#Need this for calculating log2 values
import math

#Calculate the randomness of a motif
#uses a list of dna strings as the input
#output is a number standing for the entropy
#This wasn't a very useful function.
def entropy(motifs):
    prof = Profile(motifs)
    #result = []
    result2 = 0
    for i in range(len(prof["A"])):
        temp = 0
        for key in prof:
            if prof[key][i] != 0: #don't want to take the log of zero values
                temp += abs(prof[key][i]*math.log2(prof[key][i]))
            else:
                temp += 0
        #result.append(temp) #this is really just here to see that things were working
        result2 += temp
    return result2

0.5*math.log2(0.5)*2

0.25*math.log2(0.25)*4

1*math.log2(1)

0.25*math.log2(0.25)*2+0.5*math.log2(0.5)

entropy(motifs)

9.916290005356972

In [19]:
#Identify the motif with the lowest score
#pattern is a string
#dna is a list of strings
#return a list with the best match for each dna string 
def Motifs(pattern, dna):
    motifs = []
    k= len(pattern)
    if isinstance(dna, str)==True: #verify if the input is a list or a string
        new_dna = []
        new_dna.append(dna)
        dna = new_dna #If a string, make it a list so it works with the rest of the function
    for seq in dna:
        score = k
        temp_motif = {}
        for i in range(len(seq)-k+1): #find the motifs with the lowest score
            new_score = HammingDistance(pattern, seq[i:i+k])
            if new_score < score:
                temp_motif[seq[i:i+k]]=new_score
        if len(temp_motif)==0: #if you get nothing lower, just return the first k-mer
            motifs.append(seq[0:k])
        else:
            low_score = min(temp_motif.values()) #Find the lowest score
            for key in temp_motif: #identify the motif with the lowest score
                if temp_motif[key] == low_score:
                    motifs.append(key)
                    break #just want one motif per dna string, so end search once you find it
    return motifs

dna = ["TTACCTTAAC", "GATATCTGTC", "ACGGCGTTCG", "CCCTAAAGAG"]    

Motifs("AAA", dna)

['TAA', 'ATA', 'ACG', 'AAA']

In [20]:
#Return a score for how different a pattern is from the dna input
#pattern is a string
#dna is a list of strings
#output score is an integer
def d(pattern, dna):
    motifs = Motifs(pattern, dna)
    if len(motifs)==0:
        return len(pattern)
    score = 0
    for motif in motifs:
        x = HammingDistance(pattern, motif)
        score += x
    return score

d("AAA", dna)

4

In [23]:
#Search for the k-mer that is in at least one string and has the lowest score for all strings
#dna can be either a string or a list of strings.
#k is an integer for how long the output is supposed to be
#output is median, which is a string
def MedianString(dna, k):
    kmers = []
    median = ""
    if isinstance(dna, str)==True: #If dna is a string, make it a list
        new_dna = []
        new_dna.append(dna)
        dna = new_dna 
    distance = len(dna[0])*len(dna)
    for seq in dna: #for each string in the dna list
        for i in range(len(seq)-k+1): #make a list of all kmers
            kmers.append(seq[i:i+k])
    kmers = list(set(kmers)) #remove duplicates
    for kmer in kmers: 
        if distance > d(kmer, dna): #if the differences are less than the total length
            distance = d(kmer, dna) #update the d output as the new min
            median = kmer #return the kmer with the lowest distance
        elif distance == d(kmer, dna):
            median = kmer
    return median

dna = ["AAATTGACGCAT", "GACGACCACGTT", "CGTCAGCGCCTG", "GCTGAGCACCGG", "AGTTCGGGACAG"]
dna2 = ["TGATGATAACGTGACGGGACTCAGCGGCGATGAAGGATGAGT", "CAGCGACAGACAATTTCAATAATATCCGCGGTAAGCGGCGTA", 
        "TGCAGAGGTTGGTAACGCCGGCGACTCGGAGAGCTTTTCGCT", "TTTGTCATGAACTCAGATACCATAGAGCACCGGCGAGACTCA", 
        "ACTGGGACTTCACATTAGGTTGAACCGCGAGCCAGGTGGGTG", "TTGCGGACGGGATACTCAATAACTAAGGTAGTTCAGCTGCGA", 
        "TGGGAGGACACACATTTTCTTACCTCTTCCCAGCGAGATGGC", "GAAAAAACCTATAAAGTCCACTCTTTGCGGCGGCGAGCCATA", 
        "CCACGTCCGTTACTCCGTCGCCGTCAGCGATAATGGGATGAG", "CCAAAGCTGCGAAATAACCATACTCTGCTCAGGAGCCCGATG"]
dna3 = ["GGCGGGAGGCAAGAGTCCGCGTCTCAAGATTATAAGCGGGGG", "GCGCCACACGAAGTTGGATACCCTTCGAGTGAGGGAAGACAA", 
        "TGTTTAAGGCAACCTTTTGCTTCTCAACCACGCGCTGGCTCC", "CCTATCGGACCCCGCAATGACTCAAGCCAAGTTGCGAGGGTT", 
        "ATAATATCGTTCTTAGATAATCGCACCCTCAGAGTAAGACAA", "CTCACTAGACAACAACCTAGATGCTATCTGTGGGTTACTAAG", 
        "TGCAGTGTCACAGACCGAGAGGCTCGCTCGAGCCAAAGCACA", "GAGTGGTCTCGACACCTCAGTCAAACCGCATCCTGAGCCCGG", 
        "TTGCATTCATAGGCTATCTCCTGGTTGTTACCCTACAGCCAA", "AGTCAATAGCCTCTACCGCATATAATCCAGCAGTAGGAGCGA"]
MedianString(dna, 3)
MedianString(dna2, 6)
MedianString(dna3, 6)

'AGCCAA'

In [26]:
#Similar to previous, but search kmers not in the main text (uses Neighbors)
#output is now a list of strings
def MedianString2(dna, k):
    kmers = []
    median = []
    if isinstance(dna, str)==True: #If dna is a string, make it a list
        new_dna = []
        new_dna.append(dna)
        dna = new_dna 
    distance = len(dna[0])*len(dna) #technically, an entry could poorly match everything
    for seq in dna:
        for i in range(len(seq)-k+1):
            kmers.append(seq[i:i+k]) #make a list of all kmers in the first dna string
    kmers = list(set(kmers)) #remove duplicates
    kmers2 = []
    for i in range(len(kmers)):#add more kmers that are not just in the main text
        kmers2.append(kmers[i])
        newkmers = Neighbors(kmers[i], k%2+1) #arbitrarily decided on this since no number of mismatches is stated. Essentially, adding things that can over half mismatches.
        kmers2.extend(newkmers) #add new kmers to the list
    kmers2 = list(set(kmers2)) #remove duplicates
    for kmer in kmers2:
        if distance > d(kmer, dna): #if the new kmer has a small distance...
            distance = d(kmer, dna) #reset the distance to the new min
            median = [] #reset the median to get rid of anything that performed worse
            median.append(kmer)
        elif distance == d(kmer, dna): #if distance is the same, add it to the list so we know there are multiple motifs that perform equally well
            median.append(kmer)
    return median

test = ["ATA","ACA","AGA","AAT","AAC"]

MedianString2(test, 3)

MedianString2(dna, 3)

dna = ['TGTTGGAAATATCACGTGGACCCTCTCGTGCAGTGGATATAA',
 'TGTCCGCGGCAACACCCTAGTGCCGGAGCCGGATCTGTGCAA',
 'TGGGCGTGGACGTACCCTGACCCCCAATTTCGTGAGTATCAA',
 'CTTCATAATTATAACCCTATTTTCAGTCACGAGCACCCTAAG',
 'TACCCTCCAATACAGGACGTACCTAGGTGCGCGCGCCAGACG',
 'TGTATTAACCAAAGATAGTCCCGGTACCCTGCGCGCTCGGCA',
 'AACCCTCCACTCCGGAATAATGTTTACATACCCCCTTTAATT',
 'CTGCTAGCATAGAGCATTGCTTGTTTGGCTGCCCTACACCCT',
 'GTGCAGGGCGTAAGATAGAACCCTGCTACGAGCCGTGGAAAC',
 'CAGAACACCCGCGCAAGAGACCCTTCGGTAGGAATCATGCCT']

MedianString(dna, 6)
MedianString2(dna, 6)

['CACCCT', 'AACCCT', 'TACCCT']

In [27]:
#Find the likelihood of a certain sequence occuring within a profile of dna strings (probability of each nucleotide at each position)
#text is a dna string
#profile is a dictionary with keys A, C, G, T and the same number of entries as text
#output is an integer
def Pr(text, profile):
    p=1
    for i in range(len(text)):
        p=p*profile[text[i]][i]
    return p

profile = {

    'A': [0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.9, 0.1, 0.1, 0.1, 0.3, 0.0],
    'C': [0.1, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.1, 0.2, 0.4, 0.6],
    'G': [0.0, 0.0, 1.0, 1.0, 0.9, 0.9, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    'T': [0.7, 0.2, 0.0, 0.0, 0.1, 0.1, 0.0, 0.5, 0.8, 0.7, 0.3, 0.4]
}

Pr("TCGTGGATTTCC", profile)

0.0

In [33]:
#Identify the most probably kmer from the profile thats in the current text
#text is a dna string
#k is the length of the motif (integer)
#profile is a diciontary with keys A C G T
#output is a string of length k
def ProfileMostProbableKmer(text, k, profile):
    score=-1 #start negative because you want to go as close to 1 as possible, but result could still be 0
    pattern=text[0:k] #set first pattern
    for i in range(len(text)-k+1): #cycle through the dna string
        score2=Pr(text[i:i+k], profile) #get probability of pattern occuring in the text
        if score2>score: #if probability is higher than previous, update the threshold
            score=score2
            pattern=text[i:i+k] #choose the next pattern
        if score2==score: #if score is the same, update the pattern
            pattern=pattern
    return pattern

profile = {
    'A': [0.2, 0.2, 0.3, 0.2, 0.3],
    'C': [0.4, 0.3, 0.1, 0.5, 0.1],
    'G': [0.3, 0.3, 0.5, 0.2, 0.4],
    'T': [0.1, 0.2, 0.1, 0.1, 0.2]
}

ProfileMostProbableKmer("ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT",5, profile)

text = "TGCCCGAGCTATCTTATGCGCATCGCATGCGGACCCTTCCCTAGGCTTGTCGCAAGCCATTATCCTGGGCGCTAGTTGCGCGAGTATTGTCAGACCTGATGACGCTGTAAGCTAGCGTGTTCAGCGGCGCGCAATGAGCGGTTTAGATCACAGAATCCTTTGGCGTATTCCTATCCGTTACATCACCTTCCTCACCCCTA"
profile = {
    'A': [0.364, 0.333, 0.303, 0.212, 0.121, 0.242],
    'C': [0.182, 0.182, 0.212, 0.303, 0.182, 0.303],
    'G': [0.121, 0.303, 0.182, 0.273, 0.333, 0.303],
    'T': [0.333, 0.182, 0.303, 0.212, 0.364, 0.152]
} 

ProfileMostProbableKmer(text, 6, profile)

text = "CAGCCGAAAGAGAAAGGACCAGAGTGCGCGTAGTCGGGGTTGGGGTATCCTAATTATTATTATCGGAGACATCTGGGCCCGCGCGGGACATCCAAACAGTTTAGTAGGTTTCATCTGTCTGGCCCGCCCAACGGATACGTGGCGCCGACAAACGAGTAGGGACGCGGACTTCTGCTATATCTAGTGGCCGAAATACATAAACCACACCTTGTTAGGGATCGAACCTTAGATGGCAACTGATGGAAAGATATGACTGAGGATGTCACTGGTTGCATCAGGTAGAGTTTTGGGTCGCCGGACGCCTGCGCGGAAGAGGCCGTGGCGACCTGGCGCCGCTTTCTGAACGCTCGTGGATTTTAAAACGTCTTGCCTAAAAGTCTAACATGAAGATGGTATTGGCGCAAACACTGACGAGGCTTAAGAGAATCGGCCCAGCCGGTTATACGGTCGTGATTTCTTCGTTAATTTTTCCTCGTTTTTCTGCAGCCCTGGTGGGTCAGGAAGGTTGGTCCTTAGCGTCATGAATAATGAGCTCCCATTTCGGATGTAAGATTACAATGAGCTGCCCAGCCAGAGCTTAAGAATGGATGGGGTTCCAGGTAGTGACTCCACACAGGACTAATGCAATTTTCCCGACCAGGTTAAGGTGGGATCCGTCAAATAGCTGTCTACCTCGGGACCCACTTTTCTTTAAGTTAGAATCAAGAAAGGTCGCGCAACGGATAGTATAGCCGGCTAGTAGAAAACTGCTAACTTGTCCATGAAGGAATTGCTCTAAGCGAATCATCGAAATATCCGAGGCCCATAAAACCTATAAGAGAAGAGCCTAATCATGCCGCGATTATAAGTCATGTCATGTGCATCTTTACCCTATTCTACCATCCTTAGTAGTATCGGGCATCGCACTCTACTATCGTGTATCCCGAGACCACAACCATTTAGAGTATTTTACACTCCTTAATCTACTCATGCGGCGAAGTAGCGCATGCATACAGGCGGT"
profile = {
    'A': [0.224, 0.263, 0.224, 0.197, 0.276, 0.184, 0.316, 0.105, 0.289, 0.263, 0.355, 0.25, 0.197],
    'C': [0.224, 0.276, 0.224, 0.25, 0.263, 0.211, 0.263, 0.316, 0.237, 0.197, 0.184, 0.171, 0.289],
    'G': [0.303, 0.224, 0.289, 0.263, 0.224, 0.303, 0.211, 0.303, 0.303, 0.329, 0.197, 0.276, 0.237],
    'T': [0.25, 0.237, 0.263, 0.289, 0.237, 0.303, 0.211, 0.276, 0.171, 0.211, 0.263, 0.303, 0.276],
}

ProfileMostProbableKmer(text, 13, profile)

'GCCCAGCCAGAGC'

In [34]:
#Previous counts had a problem with 0s. Made calculating probability 0 for everything.
#This will never leave something as 0.
#takes an input of the list of strings motifs
#output is an dictionary with counts of occurrences at each position, but instead of lowest value being 0, it's 1
def CountWithPseudocounts(motifs):
    count = {} # initializing the count dictionary
    k = len(motifs[0]) #get the length of each motif row (x axis)
    for symbol in "ACGT": #Make A C G T a key
        count[symbol] = [] #make an empty list for each key
        for j in range(k): #for the length of the motif...
            count[symbol].append(1) #...for the key, put a one for each position in the list for now
    t = len(motifs) #get the number of keys in the dictionary (y axis)
    for i in range(t): #for every key...
        for j in range(k): #for list position in each key...
            symbol = motifs[i][j] #for this specific position for a key...
            count[symbol][j] += 1 #Add one every time this position occurs in the matrix at this position
    return count

In [35]:
#Similar to the previous profile function, but using pseudocounts instead of normal counts
#Once again, eliminates 0s
#takes an input of the list of strings motifs
#Output is a dictionary with keys A C G T with fractions for the probability of occurrence for each nucleotide at a sequence
def ProfileWithPseudocounts(motifs):
    t = len(motifs)
    k = len(motifs[0])
    profile = CountWithPseudocounts(motifs)
    sub= CountWithPseudocounts(motifs) #setting this twice to make it less confusing for the below math
    for symbol in "ACGT":
        for y in range(k):
            profile[symbol][y]=float(sub[symbol][y])/(t+4)
    return profile

In [36]:
#Identify the most common motif from a list of motifs
#motif is a list of strings
#returns a string
def Consensus(motifs):
    k = len(motifs[0])
    count = CountWithPseudocounts(motifs) #get a dictionary for how often nucleotide occurs at each position
    consensus = ""
    for j in range(k):
        m = 0
        frequent_symbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequent_symbol = symbol
        consensus += frequent_symbol
    return consensus

In [37]:
#Score how different a motif is from the consensus sequence
#motifs is a list of strings
#returns an integer, the higher, the more different from the consensus
def Score(motifs):
    con = Consensus(motifs)
    k = len(motifs[0])
    j=len(motifs)
    value=0
    for x in range(k):
        for y in range(j):
            if motifs[y][x]!=con[x]:
                value+=1
    return value

In [41]:
#A slightly faster way to find good motifs, but not without its problems (misses things not in the sequence)
#dna is a list of strings
#k is an integer for the length of the motif of interest
#t is the number of dna strings (I was asked to write it like this, it's really not necessary)
#output is a list of strings, all of length k
def GreedyMotifSearch(dna, k, t):
    best_motifs=[]
    for i in range(0, t):
        best_motifs.append(dna[i][0:k]) #sets best_motifs to the first three k-mers in each row
    n=len(dna[0]) #how long each row is
    for i in range(n-k+1): #iterate through each row as a k-mer
        motifs=[] #set up an empty motif matrix
        motifs.append(dna[0][i:i+k]) #fill motif matrix with k-mer
        for j in range(1,t): #go through each row in the matrix
            p=ProfileWithPseudocounts(motifs[0:j]) #give a score for each nucleotide for the motif
            motifs.append(ProfileMostProbableKmer(dna[j],k,p)) #For the j row, get the motif with the highest score using the profile scores
        if Score(motifs) < Score(best_motifs): #If the new kmers score better than the artbitrary ones chosen earlier, set them as the best_motifs
            best_motifs=motifs
    return best_motifs

dna = ['GTACAAGTGAACGGCTGTTTACAGTCGGAGTAGCAGGTCTTCTGTTATTGGCGATGGCCGGTATATACGAAAGGCGAAGCGTGGTGCGATCTCTTTGCCTGTGCTCTCCGAATCCCTAGGGAGCGCAATCATAGGCTCTCCCTGCAAATGGAGCGT',
 'TAAACATAAGCCACCGGCTCCCTCTTTAAGCACAGCCCACCGCACAGAACCGATGCGTCCGATGAAATCAGTGACTGGTGGAAGTTAAGAGCTCTAGGCTGTCGACGCGCCTCGCTCTCGGTCCCCCCTTGAAGGGCCTCCCTGTAGTCAGGACAA',
 'AAACTCGCTGAGTTTAACTGCGCCGTCGTCTCTGGTTCAGGTGTGCTAAAGCCAGGTCCATTTGGCACTTAGACATGCACTCACATTTTCCTTCTGCGGACCTCCCTGTGGCACTTATAGCAAGAGGATTCACCCGAAGGACCGTCCGGAAAAATA',
 'CGGCACCTACACTTTACCCTGGGATGAGTTGCCTCATAATGTGTTGATAATGGCATATAGTCCAAACTGCCCAGGCGCTCCATGCAACTAATCGTGCATCTTTCCCTGTCTCTCCGTGCACGATAGTGTGGAACGGACAGGACGATGCGCTCGAGC',
 'GTAGCCCTCCAGAAGACTCCGTTCCAAGTTATCCGATTACCCAGAATTGTACGATAGCTACTCTGCTGCTCCCTTTTCGCGTCCAATGCGATATCGAATGAACCCGGTGGCTACATTACCTAGCCCTCCGAGTGTTTGTTACGTGGGATCTCCGTG',
 'TGAAGTAAAAGGCGGGTCTCCTTGACCGATTTAATGGTACTGTAGACGCAAGGGTGAAGCCCTGTGGCCGTAAGCACCCTACGCCAGACGCGGCATTTTGGCGGTTTTCGTTTAGGTTCTAAAACGCAACTAGCTGTTACTAGCTTCATTACAGTG',
 'GGGGGTTTCTATCTCTTCAAGTGGCCCTGTTACTCTCCAGAGGTGCGGCAAAGTCATTAACGGGACGGGCTGCATTTCCGTCTGGCTAGCATCTGGTACGGGTGCGGCTACTCAGTGGTCATAGGTTGGGTGCAATTGACCTTTAGGGCCTCCTTG',
 'TGCCTATCGGCGCTTCTGGGGAACGGGTTCTCCCTGGCTAATCTGCAGAATGAAAGGTTGCTTTTACAGAATCTTCAAGCCGGGTGCTCCAGCGACAGAAATTTCCGTGAATATCCCATATGATGTACGGTAGTAATGTTCCCCTACTTTGTTCAA',
 'TACGATCCGCACCGTTTGGGACCGATCCTGGGTGAGAGCAGGGCTAGTACGCTGCTACCAGGTCCCTTATTCGGCCTCGTCGAACACACTAACAACTAACGTGCCCTACGCCGACTCCTCCATACGGTGTACAGGCTCTCCTTGCTTCCGGAAATC',
 'GCCCGAACATCTTACTTAGCAGGTTATCAGCAAAATCGATCACTATAGAAGTTCCTGGTTAAATAGGTTAAAAGGTTCGAGGATAAGGCTTGTAAGAGTTTGCTGCTTCGGAACTCCTTGGGAGAACAGCCGCTCACGGCAAAATAACAGCGAGAT',
 'AACTTTCGATTCGATAAGTCCCAGTGTTCGATCGAGCTTGTTTTCAGCGGGAGCTCCTTGGCCCGCAGTGTAAAGCGGTGAGACACCAATCTGTTCCAGCCGAAAAGATGGTAGAAGCGGACTAGGATTGTAGCTTTTCCCATTGAAAATTCGCTT',
 'ACTGCGTTTGCCACACCCGATTTATTAGACGGCGGGTAGCAACCGTGGACGAGAAATGAATATATTTTGTCAGAGTAGGGGGCTACTCGCAAATTCTCGCTAAGGCGATGTTCAGTTCCAGTGTCTAGCATCCGTTACCGCAGACGGATCTCCGTG',
 'AATATCGCACAACTAGAGCCTCGCTTTAAAAGTCAGTAGAGGACTCCCGGGGCCTCCATGGTCTTAAAGGGATCGTTAGAGTAAAGCACGGTACGCTATAGTGCCCAGGGGGATGGTTTTAAGGGAAAGCAACCCATTTGAGGGTACTAACCGAGC',
 'TTTGCTCCATGGCGATGTCTCCACTCGCTTCTCTCTCGACCCTAGGTCCGAAGAGCGGTCTATTCCAAGGAGTGGAGCTCCTTGGTTTCTGGCGCACTTTCTCAAACAAAATATAAGTGCCCTCCCCCGGGCATACGTGAATCTTCTAGAGTATTG',
 'TGGGCCTCCATGATTCCGCTTGGCACCGTCAGTGGACTTGAAAGCCAGTGGATCAAGGGGCAAAAAGGTAAAAAGCGTATATGTGGTGAGATATTTTCTTAAAACACACTTTTGTCTACAAAGACTACGGAAATTATCTGGGTCCTCTGGAGATCA',
 'TTCTTCAGGGGAATCGAGGCGCACCGGTCCTCCTTGGTTTCGCTAGACGCGACGGGTTGTCCCTTGAGGTGCAGAGAGCATCCACTCCAGGACGACCCCGGTTTAGGCTAGTATAAACTTGAGAAGCAATGACATACATGACAAGCGTAGCGCCTG',
 'GGGTCCTCCCTGGTCTTGCACGAAGTACCAGTTATGTGCTAACGTGGCAATAGGTCCGGGCCATACGTGCTTAACATATTAATTAGGGATGTTAACGGTACGAACAAGCGAGCAGCATATGACGCCTTCTTCGGGAACTTTTATACAGTGCAGCTG',
 'CGGTCCTCCCTGTTGGGTGTCGTTCTGCTGGATTAGGCATGGACGTCCCATCGCAAGCGCAGGCTATGGTCTTCACTCGGTAAGGTACCGGCTCCCAACACTAGACAGAGTCACCTAGAGTAGAATCGCGACCGTTGAAGCGCCGAGTTTTTTACT',
 'GATTGCCAAATGGGTTGGCCCTTATTGAGTGCAAAGATACCCATCACTTTCAGTATGCGTTGGATCTCCTTGATGACTTAGAGAGAGCTTGGCTAACGTTACTTAACTGTAAACGATCACCCTGGGACCCAAATCATGTACGAAATATACGCGATA',
 'GTAAAAGAGAGCGGACGCCGTATGTTCATCATCGCGGGGACCTCCGTGTAATGTAAGCGTAAAAAGGTCGGGGTTCCCCTTCAAGCTGTATTTTCATGGAGAGCCTTCGGTATGGGGTTATGACGAGTACGGAGCATCCGAATGCTTAATGGAGTC',
 'AGGAACACTGCGAGATCGTCTATTAGGACCGGACTGTACCTAGGGATAGTTTGTATGATACAGAGGGTCCTAGACGGATATGGGTCTGGCAATCCTAGGCTCTCCGTGGGTCACCGTGGTATAATCTTTAGGCATGACATCCCTTCCGATCCCGCC',
 'GCTTGTGCACGACCGTTACGTGAATAGCCGAGTGTGTGTGGCACTGTTATAGCTTATGTACAATTCCTTATTTATACAGCCGCACGACTAGCAGACGTCCTTCCATACAGGGGCTCCGTGGGTGAGTGTTAGGAAGCTGACATAAGGGTAGTATCT',
 'TTATTACTTCGCATGGTCTCAATGGAGACTGGAAACGAGAACCGCATTCAAACTTAGTTTCACGGTGGAGTGTAACCCCACTAAGTGCATCCTGTCATATATGCATATCGGACCTCCATGCCGTCAGCACTTGTCGGACAGGATTTAGTGGTAGCC',
 'GGGGTCTCCTTGTGGGGATAAACCATAGGCAGTCGAGCAACATAACCTCTTGGCTATGATATATGTTTAGACAACTGTTGCCCCTAATGCTGGTCCCATTCAAAAGTGTATGAATCGGCTAAAATTAGTCAGGCTGTCGATTACTACCCGTAGAGA',
 'CTCGCCTCTGTCTAGACCCCTTATCGTCCTACTATAGGCATCTCGATCTCCTGGAGGAGAGAATTACTATTCTATGTATCGGGAGGATAAAGGTAATTCACCACACTTCGTCAATTGGACGGGCACTCCCTGATGGTTTGAACCACACTCATGTGA']

text = GreedyMotifSearch(dna, 12, 25)
print(*text, sep=" ")

AGGCTCTCCCTG AGGGCCTCCCTG CGGACCTCCCTG AGGCGCTCCATG GGGATCTCCGTG CGGGTCTCCTTG AGGGCCTCCTTG GGGTTCTCCCTG AGGCTCTCCTTG CGGAACTCCTTG GGGAGCTCCTTG CGGATCTCCGTG GGGGCCTCCATG TGGAGCTCCTTG TGGGCCTCCATG CGGTCCTCCTTG GGGTCCTCCCTG CGGTCCTCCCTG TGGATCTCCTTG GGGACCTCCGTG AGGCTCTCCGTG AGGGGCTCCGTG CGGACCTCCATG GGGGTCTCCTTG GGGCACTCCCTG


In [42]:
text = ['GAGCCCCATTCCGCGCGATGCTCCTTACCGAGGGTTTCGCGAAAGGCTCACATGCTAGTTAGTTCGGCGAAGACAGCTGTGGGAGTTATCCGGCCCCTGTGCCCTAAGCAGTTTTGGATTACTGTCCGATTGATGCGCAGCTCAGACTCTTAGTCT',
 'TCACGGCCTGTGTTGATCGGTCCTAGAGTACAAGTACACCATACGAAAGAATACGTCCGTGCAGGCGGAACGCCATGTGTTAAATGCCCTGAAGATGTAATAAGGGAGGAGACTCTAGTTCCCTGCCACATACCCAACGTAACCAGTAGTTCCAAA',
 'ATTCCCCGAGTCGTTTCTAAAGACCGCACGTTAATTCGCGAGGGGAGAACTAGATAATGCAGCGAAACGGCGAACGCGCTAGAAGTTGAAGGTACCCCAGCCAGCGGTAACATATAAAGAATGTAATATCGTCTTACTCTCAGGCAAAGGCTAGTT',
 'GAGAAAAGAATATGACGCCCCTTACCGGGCGCAATATTGACCGTGTATGAAAAAACCCTTTTCGTGGTGAAGAATAATCTAGTTCCCCCACTGGTTGGACCGCTTGTGGTATGTGACTTGGAACTGTTGTAAGCGAAGTTTGAGGATTGCAGCTGG',
 'AAACCCTATCAAGGAAGTTGATACGCGACGAAGGCGATCGGGTAATCAAGACCGCGCCTTCGCTCATGATATTAACGCCGAAGCCAATCGGGTAGCGGTCACAATGTACGCACTTGGTGAGACACACTAGTTTTGAGTCAACTATAGTCGGTCTTA',
 'CTGAAAGCTCGTCACAACCTAGTTGGAGTATAATATACTGGAGCCGACGGGTCTGTAAGGTTGGTACGTCCCGCGACGCCAGCTTCATTCGGACTTTCCTTCGAACTAACATTACACTTCAGGGGAATTAGCCAGGAATTCGCGACGGGAGCACGA',
 'TATAACCTAGTTGTTGCTTTACAGTTGAGGCCCTTTAACCCTAAGCCACCGGTGCACGTGTGAATGTTGCACTTCGTTTGAGGTCGCCTACGAAACCATGGTTAGAATGAGAAGAGAAACGAGTAGTGTTGGGCTTACTCGGTGCTGAGGTAGGGG',
 'AGCCTTATGTGTCACGCTCACGTCTAAAATTTCCGTACAGGCATCGCGCTGTACTGAAACCTTGCAGACAACAACACACTAGTTTATGCAGGCTGCCGCAGCTCTCCATGGTGGCGCTTGACTAGTGCCATGTCACGGGGTATGGTCCTCCTATGT',
 'AACAGGCAGTCTCTTTCTCTATCGTCAAGCTGAAGACCAACTGGGCCGCCAGTTACAAGATTATGCGGGCTCGTGTTGACATTAGGATAGTAACCGTTCTACGTAGGTCAAAGCCTAGTTGCAATCACATTCGCCGCACCCTATCACTGTGGGCGA',
 'TCATATGTCTCCGCGATCGCCGGCTTGTCGTCCTACGAGAAAAACTTTGAAAACCTAGTTGGCAGAACGCAGAGGTATCACATGGGTACCACCAAGAATAGACGGTTCAGAAAGCGGCAGAAGGGGTCTTATGCAATGTAAAACGGCGAGGTTTTG',
 'TAATGCGGAAGTCTGGTTTACCAACGTAGTCGTCGCCCGACCGCCGCGCCGTTACTACAAGAGAGTCTAGTTATAACGCTGACTTTCACCGAACTATCTGGCTGTGGAATGTCTCAGCCTTCCTCGGGAGTGTCTCGACCTTTAGGGAACGACTAA',
 'GCCCGCGTGCCAACCAGAGGTTTGTCGCGGTCCTCGCAATTGAGGAATACTGCATACTACTACATACTAGTTTGCTCTCCTCTAGGAAAGCTTGGCTTTCAGGTGCGCTAAGCGCCACTGAGTCATTAACCCAGCCACGAAGACCCTTTAAGGTGG',
 'AGACACTATTATCTGCACTCAAGTTGGAAATACTGGGCTCGAACTAACAAAACTCTAGTTTGTCACGTGTTTTCGGCCTAGCGGATCATTCAAGATTACGAATCAAGTATCTGATATGCCCAATGGTTATCTCAGTCAAAACATGACGAGTGGCAG',
 'TCACTAGAGGCTTAGATTCTAGTTACTTAGTAGTAAAGTGAGCCTATAGGGTAAGGTCCAGCCGCATTGTTTGTTCGAACCGAGCACGCTCTCGAGTATGGGTGGACGTCCTCACGGTCGCTGTGTCGAACAAAAGCAGGTAATCTCGATTTCCCA',
 'GGACTTGGAACTTTTACTGCGGCAATGTAATATATGTGGATTTCGGATAGTATGACAAAGGTACATTCGATAGAAAAGCTAGTTGACTAAAACATTTTGGCAGCGATATAGGATCTTAGATTCGCTTTAACTATATAACAAGCGTATCTTTTATGT',
 'GCTTGGGAGGGCTATATCTAAAAACCCATATGGTTTCATATACTAGTTGAGTATCCTAAGCGGTTTTTCTCATCATGGTTCTTTTGGCCCTTAAACCGTGGTGGTCAGTACTCGGATAAATCTTACGAATGACCGCGCTCCCAGATAACGCGAGTG',
 'AAGACGCTAGTTGGATTAAGCACACTTCTTGGAGCTCGCCCAACGCGATCAGAGCGTTCCCTAGCGTACGAGGGGACAGACGCTGCCGGCGAGTCGGAATTGTCCCCTTTGTAAAACCCAATCGACCTCTACACGGCGCCTAAGAGCGTTCCTACG',
 'GAGCATACTGTTGCCCCACTAGAACCTTGATAACCGCCACAAATTATGGGCGGCTGAGTGCGAGTCTGGGGAAGTAACCAGCCTCCACATCGCTGTGACAGTCTAGTTGAGCGTCTGTGGCCATATTGGTTGGTCTGGTGCGCGAGAGACACCGTG',
 'GAGTGGACACAAACTTCCGTTCCAACGACGAGATGGGTGTCTGGCACCTAGATGCTAGTTGTAGCTCTACGGATTACCCGTCCTACTAGCTCTTAATTAATTCCAGTGCGGCCACTGATGACACTAGGATGTACGCGGTTGACGGTACAAGCTATA',
 'TGTGATACTTCCATTGTACAGAATAAGTAGTTAGTTTGAGCCACTTCGTGGAGTTACGTAAAAAGCCTAGTTACTTAGGGCGGCAGAGTCCAGAGTATCATCGTTGATATATTAGAACGTGTTACGATTCCTCCGATGAGACAGGTCTTGGAACGG',
 'CTCCCGTCGAGGTCCGCGAGGTCGGGATCGAGCAAGCAACTGCCTAACACAATAGTATCGACACGAGATAACTGTTCCCGTAAAATCCCAACAGAACGAATCCCCTGGCGCTCAGCGTCCGTCGACGACCTGAGTCTAGTCCTAAACAACCTAGTT',
 'CCGGTCACGAGTAGCGATAAAGCGTTAATCTGGTAATCATCTTCAAGTACTCACCGTCCTGACATTCTAGTTTGCGGCAAGTCAGCTACCCGAAGGAAATGTACTATGGGAAGATGCCGATATTCCGGTCGAAACACCGCCAGTACTATCACTCAA',
 'TGCGGACATGGTAGCCCACAGAAACAACCTGACCTGCCTAGCTTGACTGGAAATTGTCATCAAAACCTAGTTGTTCGATATGGATTCAACAAGTAGGAGGTGAGAGGAGACCCGAAAAGCCCGGTTCCAACGTCGTTGGACAAACAATCGGTGCGT',
 'CACACCACTGTACGTTATCCCTGCCTGTTCACATAGGCGTATACCCGATCGAGCCACTCAGTCGCTCCCGGGCGAATGGTCGGTGATGACCACACCGTCGATGCAGTAAGTGGAACTGACCGAAGGTGGAGGAAGAAACTAGTTGGAAGTCGCATG',
 'TAGTACTCCGTAGTCGACGAGCCGGAGACTCTAGTTTTCCAACTAGAATTCCTGCGGAGTCCACAATCATTTGGTACCAGAATCTAGAGCAGACCTCCATTAAAATCTAACGGATCGCATATGAATGGTTTGCGTACCGCTAACAACCATATAGGG']

res = GreedyMotifSearch(text, 12, 25)
print(*res, sep=" ")

CACATGCTAGTT GAGACTCTAGTT CAAAGGCTAGTT AATAATCTAGTT GACACACTAGTT CACAACCTAGTT TATAACCTAGTT AACACACTAGTT CAAAGCCTAGTT GAAAACCTAGTT GAGAGTCTAGTT TACATACTAGTT AAAACTCTAGTT TAGATTCTAGTT GAAAAGCTAGTT CATATACTAGTT AAGACGCTAGTT GACAGTCTAGTT TAGATGCTAGTT AAAAGCCTAGTT AACAACCTAGTT GACATTCTAGTT CAAAACCTAGTT AAGAAACTAGTT GAGACTCTAGTT


In [44]:
text = ["GGCGTTCAGGCA",
"AAGAATCAGTCA",
"CAAGGAGTTCGC",
"CACGTCAATCAC",
"CAATAATATTCG"]

ans = GreedyMotifSearch(text, 3, 5)
print(*ans, sep=" ")

TTC ATC TTC ATC TTC


In [45]:
text = ['GATGGACCGGGCCATACATGGTGACACGCATCAGAAAGCTGTCCCCGTGCCTTATGCGCTGTCTGTTAGTACATCTCTCTCAATGGCCGTATTTTCAGAACAAGTATCACTTGGATCATCATCTACTCGACGGAGGGCGCGCAAGTGGGTATCTCG',
'TAAAAAGGTATAAGGGAGTCATATCCGCAGTCCTAGTGACCTTTCCCGGCCCTAGCAGTGCTCCGATAGCCCATGGATGAGACGTAACTCGGCTACTGTTTGTGACTCAAGATAGTTGCCGTCGATATCTCGGATTCTGCTTATCGTGTTACGAGC',
'AACCACGAGTACCTCTGTCGTGGTCCTTCACCAGGACTCGAAATTTGGCTCACGCCCAACGCCAAGATTACGTCGATCGTTCCTGTTGATATCTCGCCGCATATCAGGTTTATACTGATCGGCTCAGTGATTGTTAATCATCGGCGCGGGTTGTCA',
'TGTCATGTGCGGATACAGTGGTGACACCAGGTCACCTCCGACCTAAAAGCGTTCAAGGTATGGCCCGAAGAGGTAGGTATCTCTTGTGCCCGCTGCTGCTATCACTCCCTGTGAGCCCCGACACGAATGTTAAACCAGTTTATATTCGCTCGTCAT',
'AACCTAAACCCTTGCTTCCCACCATCCCTGCGAAGCAACCTTATCCGTAGTTCAGTCGCGCTGAACTCGGAAAGTGCGCTCACGAGATTCTAACTTACACTTGTTTAAGTACCACGTCCGGTGGATATCGCTGGCGTTGAAGGATAGTTCTGGTTA',
'GTGCCCGATATGCCCGTCAGGGTTGAAGTCTGGGAAGTCGTTATCCCAAACGAAATACCCCGTTCGCAGGCGTGATGTATATCCTGATTAGTACCGCGTATAATATTAGTTTCGCACAGGAGCGTGCTTGTTTTGGGTCATGGATTGGGTACAGCA',
'TAGTCTTCGAGGCATCTACCTGCGACCGAGCTTGCGATCCAAAAGCATACCCATGAAGTTTGCAAATCTTTCTTAGAGCTACAGGTAGATATCCCTGGGCCGGTAACTAGTAATATGTACAGTTTAGTTGTCCTGTACAATACTGATTGGAGTCAG',
'AGGAGACGTGTTTGGTAGGCGCTCGACCCTTACCCCCCTATCTCGACTGGCATTGGACGTCCTCAGACTGGTGGATTTGACCTAGTGGTTATCACGCACATGGGAGAACCCGGTCAGAATACATCCTGTTACAGTCAGACGCCGGGTCATTCGCAA',
'TGTGGGGATCGTCTGATTCATCGACGTCATGGAAACGGGGACGGGCCAGTGGTTATCCCATGCTGCCTTTTAGGAATTCTAGGAGGCGTTCCTAAATAGCTCGCCCGACGATATCCTACTTGATTGTGGCGGTATACCTTGCTGAACCTCGCAGTA',
'CGCAGTGCACTAGTGGCTATCGCCTTTCTATCCCTGAGGCGTTTGCGTTATAAGATCACTCTGTCGGTGTGAAACCACTGAGTCACACTGACCGGTCGACTGGCCCGTCATATGCAAGTTGAAACGCTTACTGCCGGGTATCGCTCTAAGCTCGAC',
'TACTCGTAACTTCAAAATCTCGTAGGGTCTGCAGAGTAAGAATCCCGGATACTTCAACATTATCGATTCGTCAAGACGTGCGGAGTGGATATCCCTTATTCCTATACGTCACAAGCCGCGGTCAAGTCGCTTACGCACGGTAGAGCGGGAAACCCT',
'GCTCTAGTACGCCACAGTGCCAGTACATGACCTCACGAGCCGCCACTTACGTTGATATGTTATAAATCACTAGTTTCGTTGGTACAAACAATAAGTGAGAAGCAATGAGCAACTTATCTCATAAAAAGTGTGGTCGTTATCACACATGACATAAAC',
'GAGACGGTCGTAGAAATTTGCGCTTGCTCGTATCTCAGTATCCTTCAAAGATTGAACCGGATCGCGGCGGCTAATATTGAAATCCTTAGACTTAACGTTGGTATCACTTTAGAATTTCTGACTTGAGGGTAGTGACTAGACAATCATGATGGAAGA',
'GATCGGTGGCAGTTGAATTAAGACTAGTTATCCCCTGCTTACACTTTTTCCGCCCGGACACGTGTGACGGTAGTTGATATCTCTCAAGTATCCCGACTTCACGTACGATGCCACCCATCTCCGGCAAACACACTTCTATAATATCGTGGAGCCGAA',
'GTAGGTATCACCAGAGTTCTCTCGGTATGTGGCGCTAAAACCTCTTAGAGTATGAAGGGTGAAGACCAAGCTCATACCACCCCTATATAGGGCTAATTAATTCCACATCCAGGCAAGATGTCACCCTACAGGTCGCTCACTTGTGGAGAACATCAC',
'GTGGCTATCGCTGTGATCCGTCACACAAAATCAACTTTGTAGTATTTTGGGTAGTCGGGATAACGCGTGGTCTAAGGTGAGCTGCCTTTGATCCGTTGGGTGGCTACCTTGCAAGTTACCGGCTTGTCACTAGATATAACCGGAAGTCTCTGCAAA',
'TGAGCAGACCCGACAAAGGCCATGACTTCAAAACCGGTTGTGCAGCGACAGGTACTTTAAGTACGGCACCATTAATATCGCTATACTTACGAGTTAGTGGGTATCTCATGCAAACAAACTGCTACTAGGAACTTAGACGAACTTACCAGGAGGATT',
'AGTGATCTGAGCATAGTATACTGAGAACGTGTGTGTGACCCCTCCAGCGTCCCCGGCTACTGTTCCAGATCCTAACTAATTACTGCTAGCGATCTGGTCGATATCGCTACGGAACGAAGCCGTACAATCGCCCAGAAGCGGTTAGTCAAGGGCGTT',
'CAAAATGGGAGTACTTCTCGTAGAATAATTCGTCTGTAATTCCTAGGTTCCATAGATAGGCCTCGATAGTGAAATTCCTTCATGCCCCGAGGTGGCGTTGATATCTCCTTTCTAAGCGGTACCCTTAAGAGTACTCGCGATGGGCTTATCCTCCTC',
'GTTGGTATCACCCCCAATCCTCTTAGTTTACACTGTAAGATTAACGTCAGGGTGTTGTGGATGGCTTTTCTAATTTAGGTCCTCGGGATGCTCAGGTGTTACTATCGGACTGAGTGAATGTAAGTCCGGGTATCAGCCATGAAACCACTGGAGATC',
'CCTCGGAAAACGTCGTAAGTGGTATCACAATCCACTAGTCACGGAGGGCGGTGAGTGTCTCGATGCTCAACCCCCAACCCTCAGATGAGGCTATCTGTAGGTATCACTTACGTCACTACGGGGAACAGCTCGCCTTAAGTGTAGGCTAGGTATATG',
'GGCGGCTCCATCCGATGTGACGGATTTCATGAGGCACAAGCGCTTCACTCCCTATTTGGCTCGTGACAAAGTTCAACGGCTTTGCAGATATCCATGGTCGTTATCCCGGTGGTGAACCTACCGTCAAGTCTCTACAGTGCGCGAAGTGTCCCGGGC',
'TACTAGTATAAGTTCCGAGCCAAATGGATGGCCCAGGCGCACTAGTGTTAGAAGGATTCGGGCTGTAGGTACATCAAGCTCGAATTTGTCCCACGTCATTCTGGGCCACCCGGACCACAGAAGACCTCTCTCTGTGCCGACGGTGTGGTTATCACG',
'GTGGTTATCACCGGGATCGAATACAGATACGTCTGGTACATGCCTTGTCATTATTCATACGCCCCCTGGGCCAACAATCCTTTGTCAACCGCGGTCAAAAAGGTAACTAGGATCGGCTTGATCTCTAATTCCGGACTGTTCACCACGGGTCGACCG',
'CATCACGCAATGCGAACGACTGAAGAAGGCAAGGACAGTTACGCAACCTATCATGCGTAGATCAAGGTAATCGGGACCGGTCTGGAATTTAGGAGTGTTGTTATCGCAACCTGCGATCATAACATCCTCTTATTGCCTATAAACCGACCCTGACCG']

ans = GreedyMotifSearch(text, 12, 25)
print(*ans, sep=" ")

GTGGGTATCTCG GTCGATATCTCG GTTGATATCTCG GTAGGTATCTCT GTGGATATCGCT GTCGTTATCCCA GTAGATATCCCT GTGGTTATCACG GTGGTTATCCCA GTGGCTATCGCC GTGGATATCCCT GTCGTTATCACA GTTGGTATCACT GTTGATATCTCT GTAGGTATCACC GTGGCTATCGCT GTGGGTATCTCA GTCGATATCGCT GTTGATATCTCC GTTGGTATCACC GTAGGTATCACT GTCGTTATCCCG GTGGTTATCACG GTGGTTATCACC GTTGTTATCGCA


In [46]:
text = ['AACAAAAGAGATGCCATTCAGGGGAAATCCCCCAATAGCCTGGTTGTCAACTGTGATCTCGATGTTGGAGAGTGCTGTGGCGAGGAAAGATCCCTAATAACGTATTGTCCCCTCCCTGCAACGCCGGTCAAATGGGCCCCCGCGTCAGCCCCTGGC',
'GATGGTGGAGACGACTAGGGCATTCAACTCACTGGTACTGGCCACCATGCATGGAAATACTATGTGGGGAGGCTGGACGAAGTATTCGTGACTTTCCCTCGTCTATGGGTCTCCAGGTAAACAGTGTCTACGACAGATCGGGCCTTGGTTGTGGGT',
'ACGCGCCAGACGACTTTTTACTATAGCGGGCTTATCTGAGTGTCCGAAGAAGTTGGAGAACGCTTGTGAGAGTAACAAGATATCGTTCCGAACTTAACCAGGTCCTCATTTGCACGTTAGACACGCATCTCCACCAAACCTCTTGAGTAATAACTA',
'GACGTTCGAGATGTCACGATTAATTAAGCGACTCGCAACGGAAGACCAAGGAGAATTGACAATAACACTGATCACACTGGCTTCTCTTGTTAGGACTGAAGATAGACTGCTAACTGAATAGGCTAAATTCCCGTCGGCGGGCGTGTAGCGACGACT',
'GATGGTGGAGATCGGACCGGTACCCGCCAGTATGGAAGTATTCCCCACTGACCGGACCACAATCACAGTCTGTTATCACGTAGCTTTTGAGCGGAAGAGAAGAACTGTTTAGGACTATTTTGCTGCTGACGCTAGAATTACTCGATTACCGGAACT',
'CTTACAAATGAGTCCATGAACTACATATTCCGATGGGACCGTGTATCACTCGGCTATTCGAAGACGTACCTTGACGATAGAGATTACCAGGCGAGTCCATGTACAGGTAAAGAGGCCCTAATCCCGGCCTGGACTCGGCCTTCACCGAGGGATCAT',
'CGGTAGTTTCTAGAAGCTAGAGATCCCACTCCATAGGTAAAATCGATCGCGGGTTCACAAACTTGCTACTACGATGATCCGGTTAGCCATAGCGACTGCTTAAAGTAGGGCTCTGCCCGTCCGCTTAAATTCACAGTACATGCGAGAGTCCCACCT',
'AACGCCGGGGATTGTCTCAACGAAATCTCATAACACTTTGCCTGAACTACCGGCTGACTGTTTAGGGGCCATTATCAAGCCCACGTATTAGCACGTACAAACCTAGAGGAGGGTCGAGAGTGCTCAGAGTACGTCATAACGTTTCCCTCCACAGTG',
'TTCCTATTAACGTGGCGGGAAAGACAATAAGTAATCGTTTGGATATTCGCTGTCGAATTTGAAGATGGAGAATGATCCGAGCTATTTACCCCTGGCGTTAGACCCTCAACGGGCCCATTTCGTTCGAACGTGTTTATGGGACGATTTATGTACTCC',
'GCCCCGCGAAGATGATCACCCATCAGCAAGGTTCGTCTCTGAAGAGCTCGGCATACTTAGCTTTTTTTTAGCGATGGTCGAGAACTTCTAGTGATGAGCGCAGATCATTTGTGGCCCACCTCGTCTTGAAGTGTCCCGTTTGGTTACCGCCAGGGA',
'ACCACATGCGAGAGCGTGTAGCTAGACGCTTGAGAACTTCCCGTATATCTTACATCCCCTAGACGCATACGATTTGTGACCTAACATTTAGCCGACAGAAGAACATCGATGATGCCGTACCTCGAATCTCAGACTGTGTATTACGAAGCGAGGCGC',
'GACCAATACGTCTTGTTCATCTCGAGCGTCTGGCCCCTGTGACCGACGTTATGTGTTCATGATGATGGAGAGAGACAGTTACGTGCGACGGTAGAGTCGGTCAAATGCGTTATCACACGCGAGCCATTACGTTGCCACACTTCCTTTGTCGGTCAA',
'CTCAGGGGCGCGTCCGTGTGGCAGTTGCCGAGACTTCCAGCATGTGTAGATAGTGCCTCCTGGTATGTGGGGTGTTCCATATCCGACGTTAGAGATAAATACTCCCAAGGCCCGTCCGGGCTATTGATTCATCCGTGCTCAGGGATTTGAGAGGGA',
'GATGATTGAGACACACAAGGTGCGCATGCCGTTAGGGGAAGACTGGTATTCTCTCGACGACCCGTAGGCGACGGCTCGGATTCAGAACTACAAATACACATAAGATCCTCATTTACACGGCTGTCAAATGTGACCCTTTTCCGCCGTCTGCTGTCA',
'GATGTTTGAGATCTTACACCCAATTCCCGGGACAGAGGTTATGTGCCCAGTGAAGCCTAGGAATATACGTCTATTATATTCCTCCTTGAATACCATCTTGTCACGGGTACACACCGACGGTCTCTGTGTCCTGCGTACTAGTCTCGTGAGACTCTA',
'GCGTCTTCACAGATTGCGCATCGGTCAGAACACGGAGACGGTCGAGAAGAAGGAGCGAATTCACTCTGTATAATGCCTATACGCGATCTTTCTGTTAACGGGAGGATGTTTGCTATACCTATAGCGCCTGGGAAGGACATGATTATGTTGGACAAA',
'TGCGAAGATCAGACAAGCAGCTGCGCTCGATAGGGTGGGCCCAAACGGATCTTCATTACGCCTTAAGCAGAACAAGTATTTTCGTGAAAACGGTTGTTACACGATGTCGCTCGCTGCCGCAACGGCGATAAGACGTTTGTGCTGGACGCTAGAGAC',
'ATGAGATCGGATCTTTTTCACTGGGATCCATCTCCTTCTTGGATTTATGTAGGCTTAGTATTGTGATTCCCTAGCTAGTCTTATAAGCGACGTCCGTGTCAGGCGTCGGCTCTTTATTGTATAGGCCCGCGTGAAGATAGAGAGACAGTTCAGTGT',
'TTCTGAGGCACGGACCCAGGGTCACCATAGCTTATAGGGACCGAAATAGAAGTTCGAGAGAGCCCTGCATCTCCTAGGAATAGATGCACAATTTCGCCTGCTTGTTAGGAGCTTTAACGACCTTCCCAAGTGGAGTCGGTCGTAGAGTACCGACAC',
'ATAGTACTAGTTTCTGAGTGGCCGTATTGCCGTGCAACATGTTAGATAGAAGATAGAGACAATGGTGGAAATTACGGACTGAAACAGGGACTATCAGTATAGTTGTTAGTCCAGTCATAGGCTGCTTTGAGATGCATTCATCGACCGGCGGCGGTT',
'TTTAATCCGAACTATCTCAAAGCTGAGGTTCGAGATCGACTATGAGAACACCTTAATTCGTGGTGTCCATTAAAGCACCAACAACCGGAGATACCTGAAAAATGTCTGTTCATGCCTCGGATTCCGGATACTCCCTTTTATAAGACTGCCGCTCTC',
'GTACCACAGTGGCTCCGTTACCTCGAGGATGGAGATTCCCAAACAGTTTCAGAACGCGAGCGAGGCGAGACTCCAAGAGCAATAGTTGCCTCCCGTCGTTAAAATGCAATGGTAAACCAAAGTATATAGAGACTGTATATGTACTCTGTTTTGAAT',
'GCAACTTAGATCCGTCTGCAGGCGGAGGTTGGAGAGTTGGGCCTGGGATAAGGCTGCTCTCAGCGTGGGTGCGGCGGGTAGCAGGGATCCGTATACTGCATTAGTATTGATCATATGCTAACATCCCTGCGAATAATAACTCCACCGTCATAGGCT',
'GAGGCTAGAGAATTGAACTAAAGCGTCCTTCGCGTGTACCTGATAAAGCCCAGTCCGACGCCCGCGCAGTTGGCACGACGGGCACATTAGAAAAGGCAAATGTGTACATCGGCTTGTTATGTGAACCCTATGCGGCTTCCTGTTTGAAATCCCCTT',
'ATCTATGAATTGAGCCTTTACGTATGACGGAATCAGGTACAGGCACCCAACCACTGCATACAGTATCTGACTAATGTCTGGCAGTCACTTCTTACTGACGGTAGAGAAGACCACAAGTCAAGATTGGAACATCGAGAATACTTAGGGATTCAGCAA']

ans = GreedyMotifSearch(text, 12, 25)
print(*ans, sep=" ")

GATGTTGGAGAG GATGGTGGAGAC GAAGTTGGAGAA GACGTTCGAGAT GATGGTGGAGAT GACGATAGAGAT GAAGCTAGAGAT GAGGGTCGAGAG GAAGATGGAGAA GATGGTCGAGAA GACGCTTGAGAA GATGATGGAGAG GACGTTAGAGAT GATGATTGAGAC GATGTTTGAGAT GACGGTCGAGAA GACGCTAGAGAC GAAGATAGAGAG GAAGTTCGAGAG GAAGATAGAGAC GAGGTTCGAGAT GAGGATGGAGAT GAGGTTGGAGAG GAGGCTAGAGAA GACGGTAGAGAA


In [50]:
text = ['TATAACTGATGCGGTTACGCCCATCCCAATCTACTGTCGCAGATAGCGCGGTACGTTGGGTTTCAATTTGACCACATCGCCTGAGGCACTTGTAGCTTCACAGGTATCTCTATGTTCCCCTCTAGCTACCGAGCACCTGACCGAGGCATGCGAAAG',
'CCACGTGCTTGTTATCGTGAGATCCGAGTGGTTACCGTGGCTCTAGTATGGTGAGAGTAGGGATACGCCCTTAGGTTGCACGCAACTTACCCGGCAGTCAACAAGTCATCACGTCCACGATGAAGTAGTTTGATCATCTTACGAAGTTGTGGTGGC',
'AGAGTGCGGGACCAGCCCTCTCCATTACTATGCTGAGGGGACGCCCCTCCTCGTAAGTTAAGGAAATGCGGAGACAGACGATAATCAGTTTGAGATTAGGTGCTATACGGGTCATGCCGCGTAGGACACGAACATGTAGACCCAAAAACATGTGTA',
'CGAGCGGATGACGAGGGAGGCCTGGAGATAGAAAATACCGTAAGTATGGACTATGTGGGTGGCCACGCGCCTTAGTTGGAGCTCGGATTCTCTGTTGAATAGCTGAAACCACTCGGGATACGAGGGTGTGACACACCGTCTTTGGGTGAAACCGTG',
'GTCTAGTAACCGTCAAATCGCCCATCCTCATTCTCAAACCACCTGCCCAGTATCATGACTCGGATTTAAAAGTCCCCCGAAGCCTGATATTCCAAACCTTATGGGTAGGGACACGCGCGTGTGTGACTGTACATACGCGCGGATTATGTGTAACAA',
'TTGAGACACAGCAATTCACCAACTATTCCCCCTTTTTGATTTGTTGGCGGCAACGCGCCTTCATAATAGTTCACATCCACCAGCTCAATACAAACACGTGTTTCGAACACACCCAAACGTTGCCCCTCCAACGCTAACTGGGCCGGACCGGCCGCG',
'CAATTTAATTGTTATGATATAGGAGCGAGTTGTCAGCCGTTTCTCAGCTGACACGAGGCAGGATACGCCCTTCACACTGTCCGTGAGGTCACGTTAACTTCTCGCCAGAAGGTAAACCTCTACGTGATCATTTGAATTTCTGAATCCGGGATACTG',
'GCAGAACAGTTCTAAGAAGTTATATTCTAAATCCGTTTAATGCATAGCTGGGCTCAAGAACTCGCCTGCCTTAATGAAGCTAGTAGTTGGGATGAATGCGACAAGATACTTTGATGAGAAGGCAACGCTCCTTCGTGAAGGACGTCACAATACTGG',
'TCTGGTGACAAGCACTGGTAAGAGGCGGCCGGGCCAACCTTATAGGAAGGGCACGCACGTGCCGGCGCAGCAGAAACTGTTTAGGGTGGCTAACTATCCCGGAGCTTATGGTGAATCCATGAAATCGGGCAACGTTAAGTGCCTGGGTAATTGTCG',
'GACCAGGTGTAAGTCTTCTTAAAGACGGCCTCGCTGAGGTTTTAAAGAACCAGATCGTCGGTCTGTGAAATTTTTATTCGATGCCTCGAACTGTTCCTCACCCCGACTGGAAACGCACATGCCATTGTACTAAGAATGAAAACTTGCCGTTGATTT',
'CTAAGCTACTGTAAACAATAACTGGGGTGCCCGGGTGGTTCCCACTTTCATCGGGTGACTTCTACAAACTAAATTTGATATATAGGCTACGCCCCTGACAAGCCGGACTTTCGGTCGCTTAGAAGTCCGCTTGGAGGGAAGACAGGGTTTATGCGG',
'AGCTCGAACATCGATGAGCAATCAGATAGAGCACCCGGTCCCCTAACCCACACAGAGCCCGGGCACGCTCTTCCAGGCTAGTTATTTCGAGGTTTTCTTTTTTATAGTGTGGAGACCGGAAAAACACACTACGATCCGCAGTCGTCGGACAATTAA',
'TAGGGCTGAACTTGGGCAATCCGGTCCGATCAGTGCTCAAAGGGTAACGATATCGTACGTGACGCGGTGTAAATTAAACCTCCTCGGCCAACATGAGGCTACGCGCTTAGGGATTGGGTTCAGCATCCTGAAGTTCATTGTCCACTAACGCACTAC',
'CGAAGTACTCGATTAAGACACGTTCAACATCGCGATGAAAGTCTTGGAACGAGGAATAGGGGTCACGCCCTTCCAATATCAGAGAGGAGCGAGACCTGAATACTAAAAACAGGCCCTGTAGTGTAGTAGACGCGTCCGTGTTGCTAGATATTGCGT',
'AGGACTAACAGTGTATAATGGGCCTCAGAGTTACCGTTGGACGCCCTTGCAGGTCCCGACATATGCTGAGGAGGAAACGCTCGTGATGTCATCCTAACGACATCTCCAACACCCGTCGGCTTTTTTTAATAGAAAGCTATTCGCTTATGGTGGGAT',
'GACGGCACACTTAAGTTATAGTGGGGCAACTAGGTCGAAGTCCTGTCTCCCCAAGTATCTTTATTTCCCACACAATGGAGTCTAGAATGGCCTCCAGGAGACGCCCTTTTTATCCCGACTGATGCACAATTGTTGCTTGCATATGCTACAATATAG',
'CGAGGACGAGGAACGAGACCAGTATTCGCACTATTGTTGAGTTCTTTAGGAAACGCTCGTCTTGTCCCGGAGAGTCATGTAACTGCTACGGGCCAGGCACCTTCCATCGGGCCAGACGTTCATACCTCCGCTGGTATTCATAGAACACTTACCGCT',
'AGCTCTTCTACACCTTGATAGCCTAGCTTGCTATGGAGGCCCAAGTCAGGAGACGCGCGTTCAATGTGTATCTGGCCCACGATATGCATACAGGAGACTCTACCTCTTTAACGTATACTACGACCTGATGAATTTGGGGGGTGGCTCGGGGTGACC',
'AGTTAGGATCCCATTGCATCTTTGACGTCGTCAGATATGGTAGGTACTCGGTTAACGAGGACCCGTGGTCTAATGCCTTGAAACGAAGCTAGCGGATGTAATTCCCGCAGAACTAGACTAAGCCCTACCTATATTAAACGAACAGGCGACGCACAT',
'ACAGTTGGCCATGCGCTGACTGACATAGGGAGAATCAGTCGGTGAAAGCACCGTGACGCCGTACCGCCATGGTACTATGGAGAGCCAAATCGTATGAGTAAAGGCCCCGGGGACTTGGGGAACGTATTGGATGGCCACGCACCTTAAAACAGTTCC',
'TAATTACCTGTATAACGGTTTAAGAAATTTGTCTTAAGACAAGTATTAGCCGAGGTTTTGGGTTACGCCCATCTAACTGCGGGTACACGCGGAACAAGGGGGGACTAGATGTTGAGTGATGCCCGCGAAGCCTCCCCTGCAGTGAAATGACTGTGT',
'TGAGGGTCCGGGTCCAAGTGGACAGTTCGTCGAGCACCCGGCACGAGCTTGTATGCGAGCCAGGGTTGCAATGTGCTTAAGGAAGGTTACGCACGTAGCTTCACTGCAACCAGAGGCCTGGAGGGGACGATATAGTCTGTTTTGTGTAGTCTGATG',
'TCTGCGTTTTGGAACACGACTTCCTGTTAGGGTCGTGCCTCACGTCTTGGCTACGCGCATCATGTACGACGGCGAAGGCGCTTCGCACACCCTTCCTTGCCGAGCAATAGACCATGCCGGCTTGTCTCGCGCAGTGACCGTCCTGTGCCCCCTCGG',
'TCGCACCGAATGAAGCAGGGCGACTTTTGCCTAGCTGGGAACGCCCCTCAGTAGCTGTTGACTGGGGAAGAGGCCCACTGTGGCCAACACGCTGGAAGTTTTTTCCTGTTAGTGTGCTATGCAGATCATTTAGTTTTAACTGGCCCAATCTCCTGA',
'GTAGCTTATGCAGCAACATCGTCTATAGACTAACAGGTTGAGTTAGGAGGGTACGCACTTCGGAACAGTGCAGTCTGTTAAACTTCAATACCCCCTCGGTACTTCCGCGTTTTCCAATAGCTCATAAGTCACTGAACTTCTGGCGAGTCCTTGGCT']

ans = GreedyMotifSearch(text, 12, 25)
print(*ans)

GGTTACGCCCAT GGATACGCCCTT GGGGACGCCCCT GGCCACGCGCCT GGACACGCGCGT GGCAACGCGCCT GGATACGCCCTT GGCAACGCTCCT GGGCACGCACGT GGAAACGCACAT GGCTACGCCCCT GGGCACGCTCTT GGCTACGCGCTT GGTCACGCCCTT GGAAACGCTCGT GGAGACGCCCTT GGAAACGCTCGT GGAGACGCGCGT GGCGACGCACAT GGCCACGCACCT GGTTACGCCCAT GGTTACGCACGT GGCTACGCGCAT GGGAACGCCCCT GGGTACGCACTT


In [60]:
def profileMostProbable(text, k, matrix):
    probKmer = text[0:k]
    highestProb = 0
    for i in range(len(text)-k+1):
        kmer = text[i:i+k]
        probability = 1
        for a in range(len(kmer)):
            symbol = kmer[a] 
            j = 0
            if symbol == 'A':
                j = 0 
            if symbol == 'C':
                j = 1
            if symbol == 'G':
                j = 2
            if symbol == 'T':
                j = 3
            
            probability = probability * matrix[j][a]
        if probability > highestProb:
            highestProb = probability
            probKmer = kmer 

    return probKmer

def SymbolToNumber(symbol):
    if symbol == 'A':
        return 0
    if symbol == 'C':
        return 1
    if symbol == 'G':
        return 2
    if symbol == 'T':
        return 3

def formProfile(motifmatrix, k, t):
    countMatrix = [[0] * k for a in range(4)]
    sumOfCol = 0
    for col in range(k):
        for row in range(len(motifmatrix)):
            text = motifmatrix[row]
            symbol = text[col]
            num = SymbolToNumber(symbol)
            countMatrix[num][col] = countMatrix[num][col] + 1

    for row in range(4):
        sumOfCol += countMatrix[row][0]

    profileMatrix = [[0] * k for a in range(4)]
    for row in range(4):
        for col in range(k):
            div = countMatrix[row][col]
            profileMatrix[row][col] = div/float(sumOfCol)

    return profileMatrix

def findScore(motifs, k): 
    score = 0
    for col in range(k):
        col_i = []
        for row in range(len(motifs)):
            text = motifs[row]
            col_i.append(text[col])
        freqArray = [0, 0, 0, 0] 
        for sym in col_i:
            symNum = SymbolToNumber(sym)
            freqArray[symNum] = freqArray[symNum] + 1
        maxFreq = max(freqArray)
        score_i = sum(freqArray) - max(freqArray)
        score += score_i

    return score

def GreedyMotifSearch(dna, k, t):
    bestMotifs = [] 
    for i in range(t):
        text = dna[i]
        kmer = text[0:k]
        bestMotifs.append(kmer)

    for c in range(len(dna[0])-k+1):
        motifs = ['']
        text = dna[0]
        motif = text[c:c+k]
        motifs[0] = motif 
        for i in range(1,t):
            profileMatrix = formProfile(motifs, k, t)
            newMotif = profileMostProbable(dna[i], k, profileMatrix)
            motifs.append(newMotif)

        if findScore(motifs, k) < findScore(bestMotifs, k): 
            bestMotifs = motifs

    return bestMotifs

def main():
    with open('dataset_159_5.txt', 'r') as myfile:
        data = myfile.readlines()
    numbers = data[0]
    numbers = numbers.rstrip('\n')
    nums = numbers.split()
    k = int(nums[0])
    t = int(nums[1])
    dna = []
    for i in range(1, len(data)):
        text = data[i]
        text = text.rstrip('\n')
        dna.append(text)
        
    bestMotifs = GreedyMotifSearch(dna, k, t)
    for motif in bestMotifs:
        print(motif)
        
bestMotifs = GreedyMotifSearch(dna, 12, 25)
print(*bestMotifs)

AGGCGAAGCGTG TAAACATAAGCC AAACTCGCTGAG CGGCACCTACAC TAGCCCTCCGAG TGAAGCCCTGTG GGGGGTTTCTAT GAAATTTCCGTG AGGCTCTCCTTG CGGAACTCCTTG GGGAGCTCCTTG CGGATCTCCGTG TAGAGCCTCGCT TGGAGCTCCTTG TGGCACCGTCAG AGGGGAATCGAG GGGCCATACGTG TGAAGCGCCGAG TGGATCTCCTTG GGGACCTCCGTG AGGCTCTCCGTG AGGGGCTCCGTG AGAACCGCATTC GGGGTCTCCTTG GGGCACTCCCTG


In [61]:
#Find the best motif according to a profile and a list of dna strings
#profile is a dictionary with keys A C G T
#dna is a list of strings
#output is a list of the best motifs for each string
def BestProfileMotif(profile, dna):
    kmer=len(profile['A'])
    pattern=dna.copy()
    for j in range(len(dna)):
        score=-1
        for i in range(len(dna[j])-kmer+1):
            score2=Pr(dna[j][i:i+kmer], profile) #probability of occurrence, want this to be as close to 1 as possible
            if score2>score:
                score=score2
                pattern[j]=dna[j][i:i+kmer]
            if score2==score:
                pattern[j]=pattern[j]
    return pattern

In [62]:
DNA = ["TGACGTTC", "TAAGAGTT", "GGACGAAA", "CTGTTCGC"]
test = ["TGA", "GTT", "GAA", "TGT"]
profile = ProfileWithPseudocounts(test)
BestProfileMotif(profile, DNA)

['TGA', 'TAA', 'GGA', 'TGT']

In [63]:
#need random number generator
import random

#Make random motifs of length k
#dna is a list of strings
#k is how long the desired motifs will be (integer)
#t is how many strings are in list dna (instructed to include this, I feel like you could easily leave it out and include it in the function)
#output is a list of strings length k
def RandomMotifs(dna, k, t):
    sub = []
    for i in range(t-1):
        x = random.randint(1, (len(dna[i])-k))
        sub.append(dna[i][x:x+k])
    return sub

In [64]:
#Start with random motifs to get closer to the best motifs
#Randomized way is faster, but not really as reliable...
#dna is a list of strings
#k is length of motif (integer)
#t is how many strings are in the list dna
#output is a string of length k
def RandomizedMotifSearch(dna,k,t):
    m = RandomMotifs(dna, k, t) #generate random motifs firt
    best_motifs = m
    while True:
        profile = ProfileWithPseudocounts(m)
        m = BestProfileMotif(profile, dna) #identify the best motifs out of the random motifs according to profile
        if Score(m) < Score(best_motifs): 
            best_motifs = m
        else:
            return best_motifs

In [71]:
prac = ["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
"GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
"TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
"TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
"AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]

res = RandomizedMotifSearch(prac,8,5)
print(*res)

prac2 = ["GGCGTTCAGGCA", 
         "AAGAATCAGTCA", 
         "CAAGGAGTTCGC", 
         "CACGTCAATCAC", 
         "CAATAATATTCG"]
res = RandomizedMotifSearch(prac,3,5)
print(*res)

GTGTTCAG GGTGCCAG GTATACAG CACGTCGG TCCACCAG
TCG GCG GCG TCG GCT


In [72]:
#Randomized motif search gets better over many iterations
N=1000
score1=5000
for i in range(0,N):
    if i%10==0:
        print(i) #This is just so I know where I am over the iterations
    tempmotif=RandomizedMotifSearch(text,15, 20)
    score=Score(tempmotif)
    if score<score1:
        score1=score
        best_motifs=tempmotif


print(*best_motifs, "\n")

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
CGGTTACGCCCATCC GGGATACGCCCTTAG AGGGGACGCCCCTCC TGGCCACGCGCCTTA GGGACACGCGCGTGT CGGCAACGCGCCTTC AGGATACGCCCTTCA AGGCAACGCTCCTTC AGGGCACGCACGTGC TGGAAACGCACATGC AGGCTACGCCCCTGA CGGGCACGCTCTTCC AGGCTACGCGCTTAG GGGTCACGCCCTTCC AGGAAACGCTCGTGA AGGAGACGCCCTTTT AGGAAACGCTCGTCT AGGAGACGCGCGTTC AGACTAAGCCCTACC TGGCCACGCACCTTA GGGTTACGCCCATCT AGGTTACGCACGTAG TGGCTACGCGCATCA TGGGAACGCCCCTCA AGGGTACGCACTTCG 



In [80]:
import random

# Input:  String Text, an integer k, and profile matrix Profile
# Output: String of most probable pattern
def ProfileMostProbablePattern(Text, k, Profile):
    n = len(Text)
    maximum = -1
    probable_pattern = ''
    for i, letter in enumerate(Text):
        for i in range(n-k+1):
            pattern = Text[i:i+k]
            probability = Pr(pattern,Profile)
            if (probability > maximum):
                maximum = probability
                probable_pattern = pattern
    if maximum == -1:
        return Text[0:0+k]
    else:
        return probable_pattern

In [81]:
# Input:  A set of kmers Motifs
# Output: CountWithPseudocounts(Motifs)
def CountWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    count = {}
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
            count[symbol].append(1)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

In [82]:
# Input:  A set of kmers Motifs
# Output: ProfileWithPseudocounts(Motifs)
def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    profile = {}
    count = CountWithPseudocounts(Motifs)
    for key, motif_lists in sorted(count.items()):
        profile[key] = motif_lists
        for motif_list, number in enumerate(motif_lists):
            motif_lists[motif_list] = number/(float(t+4))
    return profile

motif1 = "AACGTA"
motif2 = "CCCGTT"
motif3 = "CACCTT"
motif4 = "GGATTA"
motif5 = "TTCCGG"
motifs = [motif1, motif2, motif3, motif4, motif5]

print(ProfileWithPseudocounts(motifs))

{'A': [0.2222222222222222, 0.3333333333333333, 0.2222222222222222, 0.1111111111111111, 0.1111111111111111, 0.3333333333333333], 'C': [0.3333333333333333, 0.2222222222222222, 0.5555555555555556, 0.3333333333333333, 0.1111111111111111, 0.1111111111111111], 'G': [0.2222222222222222, 0.2222222222222222, 0.1111111111111111, 0.3333333333333333, 0.2222222222222222, 0.2222222222222222], 'T': [0.2222222222222222, 0.2222222222222222, 0.1111111111111111, 0.2222222222222222, 0.5555555555555556, 0.3333333333333333]}


In [83]:
def GreedyMotifSearchWithPseudocounts(Dna, k, t):
    BestMotifs = []
    # search through DNA string
    for i in range(0, t):
        # starts by setting BestMotifs equal to the first k-mer from each string in Dna
        BestMotifs.append(Dna[i][0:k])
    n = len(Dna[0])
    # ranges over all possible k-mers in Dna[0], trying each one as Motifs[0]
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            # builds a profile matrix Profile for this lone k-mer, and sets Motifs[1] equal to the Profile-most probable k-mer in Dna[1]
            P = ProfileWithPseudocounts(Motifs[0:j])
            # sets Motifs[i] equal to the Profile-most probable k-mer from Dna[i] based on this profile matrix
            Motifs.append(ProfileMostProbablePattern(Dna[j], k, P))
        # GreedyMotifSearch checks whether Motifs outscores the current best scoring collection of motifs, BestMotifs
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

In [84]:
def Consensus(Motifs):
    k = len(Motifs[0])
    profile = ProfileWithPseudocounts(Motifs)
    consensus = ""
    for j in range(k):
        maximum = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if profile[symbol][j] > maximum:
                maximum = profile[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

In [85]:
def Score(Motifs):
    count = 0
    consensus = Consensus(Motifs)
    for motif in Motifs:
        for index, letter in enumerate(motif):
            if letter != consensus[index]:
                count += 1
    return count

In [86]:
def Pr(Text, Profile):
    p = 1
    # loop through each index(char) in text
    for index,char in enumerate(Text):
        for key, profile_lists in sorted(Profile.items()):
            if char == key:
                p *= profile_lists[index]
    return p

In [92]:
k = 3
t = 5
Dna = ["GGCGTTCAGGCA", "AAGAATCAGTCA", "CAAGGAGTTCGC", "CACGTCAATCAC", "CAATAATATTCG"]
res = GreedyMotifSearchWithPseudocounts(Dna, k, t)
print(*res)



TTC ATC TTC ATC TTC


In [96]:
k = 3
t = 5
Dna = ["ACGAGAACTGTAATGGAGACCAATCGGGTCGTATGTACGACACGGATCTTCTGTATCGATCATCGCTTAACTTATACGATCTCATTCTCACGACGATCCTCAACCCCGGATACCCGCACTGCCTCAATCCGAAGTACTGCGTAGTACTTTACCCTT",
"ACCTGTGATCACTCAGAGAACAGAGGATCCGGGTTGGATGTCAGTGTTATGCCAAGAAACGAGACCTAAGGTGCCGTCCCCGGCGGAATGCTTCTCGCTTCCCCTTCTAAAGGGTCCTGGCAAAATGCTTGTGACTTTGAATGCCCTCACTGAACT",
"AAATGTGAAACCTATATCAGTCATTATACCGGCGCGATGTTAACTCGCACCTGATTGCAAGGTCACTGATCGCGTCACTACACTAGAGTTTATTCATACCTGCATGGGGGGCATGATGGATGAATTTTAACTAGGTGATCGTGACCAATGTTCACC",
"TTTAACCGAATGAGAAGGGTTTCGTTTTAGGCCGTCGATCCGCCGCTTCTTCTCTCGACTGAATCGGAGGTTTTAATGTGTGTGCTAAAGTGAACTGCCTAATAACCCGCTAGGGGGATGATTTATTGTATCTGCAATGTACGGTGGTACATCCCG",
"GCTTTCCCGTGTTCTTCGTGCGCTCGGGACCAGAGATCAGAACTAGTGCTAAAGCCATGGAAGCATTTAGCCGGCCGATGTAGTAAAGTTGCCCATATTTCTCCCTAAACCAGCGTATACGTCGAAAACTTTCTTCACTCCACTATGTATAGATGG",
"CGCTTAGCGCATCCTCGCTAAACTAATTTGTAGGAGCAATCGCATGTCGACACCGAGGAAGACAACAAGTACATTAGTTACACGCTTCTTACTGGGGATAAAAATCTAGGATCGCGTGATCGGCGTGCTCCGGCGTCAGGTGACTTGATGGCCCCA",
"AGTCATGACAATTCGCACGAGGTGACTTTCAATCGACTTACGTACTGCGCGTCGATGTCGCTTCACTCCACTAATACCCATATTCCTAACACGCCAGTACGGTTCAGAGTCGGCGGCTGAGGGGCCCTAGAAACGAGACACCCTAGAGGCTCTGGG",
"ATAATAAACGAGATTTAGTGCTCCAGGTCCCTCTAACTTCGCTAGACTGTTCACGGTACTTAGAGTAGCTCAGAAATCGCCTGTCTTCGGGTTGGTTTTTTCAGGAGGTGCTCTGTGCGTTAACATACCAAGCCATAGCTGCTTTTCCTCTACAAA",
"CCTACCGGAGGACTTCACTGAACTAAGTACTGGGGTGCTAAGGTCGAACGAGATGACTGGACCCTACTCTCTACGGGACCGACGCCCCAGGGCTTAATTCATATTGACTAGATTTATGATAATAATAGACTCGGCGGTTTGTAGCTTTCCCCTGAA",
"CATTTCTATAAAGCTACAATAATAATCCGCGCTGTCGGCAGACGTGGTACCGACCCTACTCCTACCGTTTGAGAGATGGAGGGTCTTCCCTGAACTAACGGCATGCATGAGAGGGGTACGACCCTGGTACTTCTGAAACCAGCATCCGCGGCGACG"]

res = GreedyMotifSearchWithPseudocounts(Dna, k, t)
print(*res)

CGA CGA CGA CGA CGA
